# Tract/Patch LSDB Search Fn: Looking for skymaps used in LSST 

Intro borrowed from: https://github.com/lsst-sitcom/linccf/blob/main/internal/2025-02/skymap_fun.ipynb

In [4]:
import lsst.daf.butler as dafButler

In [38]:
!butler query-collections /repo/embargo "*/DRP/*"

                                                                       Name                                                                            Type   
-------------------------------------------------------------------------------------------------------------------------------------------------- -----------
LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409                                                                                           CHAINED    
  LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409/20250430T152328Z                                                                        RUN        
  LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409/20250429T180628Z                                                                        RUN        
  LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409/20250428T211144Z                                                                        RUN        
  LSSTCam/runs/DRP/20250415_20250422/d_2025_04

In [39]:
config = "/repo/embargo"
butler = dafButler.Butler(config, collections="LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409")

In [40]:
butler.registry.queryDatasetTypes("sky*")

[DatasetType('skyMap', {skymap}, SkyMap),
 DatasetType('sky', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True),
 DatasetType('skyCorr', {band, instrument, day_obs, detector, physical_filter, visit}, Background),
 DatasetType('skyCorr_metadata', {band, instrument, day_obs, physical_filter, visit}, TaskMetadata),
 DatasetType('skyCorr_config', {}, Config),
 DatasetType('skyCorr_log', {band, instrument, day_obs, physical_filter, visit}, ButlerLogRecords)]

In [41]:
refs = butler.registry.queryDatasets("skyMap")#, collections="LSSTComCam/raw/DP1/DM-48240")
list(refs)

[DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'DC2_cells_v1'}, run='skymaps', id=b7d4b4f1-e108-4570-bbf5-da6093179ac8),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'hsc_rings_v1'}, run='skymaps', id=6e7fc928-0dc3-4e1b-ae63-5fb63be46216),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'latiss_v1'}, run='skymaps', id=f677ff86-2b1c-4094-9dad-e39395052cd1),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'lsst_cells_v1'}, run='skymaps', id=857e0e80-8d08-44f4-a81b-84227b064d5b),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'ops_rehersal_prep_2k_v1'}, run='skymaps', id=8f5f7e7a-c647-4a88-b8e7-6e2406d071b8),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'discrete_comcam_rgb_ecdfs_v1'}, run='skymaps', id=1b399fae-8a7e-4633-833d-b713a93d44a3),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'discrete_comcam_rgb_seagull_v1'}, run='skymaps', id=ca37ba9c-f4ff-4ab1-a27b-7930f4f7739b),
 Data

In [51]:
skymap = None

for ref in refs:
    if ref.dataId["skymap"] == "lsst_cells_v1":
        skymap = ref
        break

skymap

DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'lsst_cells_v1'}, run='skymaps', id=857e0e80-8d08-44f4-a81b-84227b064d5b)

In [55]:
butler.get(skymap)

In [56]:
skymap_uri = butler.getURI(skymap)
skymap_uri

ResourcePath("s3://embargo@rubin-summit-users/skymaps/skyMap/skyMap_lsst_cells_v1_skymaps.pickle")

In [58]:
import os
from lsst.resources import ResourcePath

In [62]:
target_dir = "/sdf/home/o/olynn/"
base = ResourcePath(os.path.join(target_dir, "skymap"), forceDirectory=True)
new_uri = base.join(skymap_uri.basename())
if os.path.exists(new_uri.path):
    print(f"File {new_uri} already exists. Skipping download.")
else:
    new_uri.transfer_from(skymap_uri, transfer="copy")
    print(f"File {new_uri} downloaded successfully.")

File file:///sdf/home/o/olynn/skymap/skyMap_lsst_cells_v1_skymaps.pickle downloaded successfully.
